## **Import required modules**

In [ ]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient  # import the respective chat completion client according to your model
from azure.identity import AzureCliCredential, get_bearer_token_provider, DefaultAzureCredential, ChainedTokenCredential

from autogen_ext.tools.mcp import  StreamableHttpMcpToolAdapter, StreamableHttpServerParams
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken


import nest_asyncio
nest_asyncio.apply()

## **Connecting to MCP Server**

In [ ]:

# Create server params for the remote MCP service
server_params = StreamableHttpServerParams(
    url="http://127.0.0.1:8000/mcp",   # update it with deployed url if available
    headers={"Authorization": "Bearer your-api-key", "Content-Type": "application/json"},
    timeout=60,
)

# Fetcht the respective tool adapters from the MCP server
video_agent_tool_adapter = await StreamableHttpMcpToolAdapter.from_server_params(server_params, "video_agent_tool")

## **AutoGen Model Client Creation**

In [ ]:
credential = ChainedTokenCredential(
    AzureCliCredential(), DefaultAzureCredential()
) 
token_provider =  get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

# Create an agent that can use the MMCTAgent's tool
model_client = AzureOpenAIChatCompletionClient(
    azure_endpoint="https://<your_resource_name>.openai.azure.com/",
    azure_ad_token_provider=token_provider,
    api_version="<api-version>",
    model="<model-name>"
    # api_key="api-key"  # if authorization is not via managed identity
)

## **Create an Assistant Agent**

In [ ]:
agent = AssistantAgent(
    name="video_agent",
    model_client=model_client,
    tools=[video_agent_tool_adapter],
    system_message="""You are a Video Agent whose role is to perform relevant tool calling and return answer according to user query.""",
)

In [ ]:

await Console(
    agent.run_stream(
        task="""
query: <user-query>,
index: <valid-index>,
 """, 
    cancellation_token=CancellationToken())
)